# Хакатон: Прогноз доходов (Final Optimized Pipeline)

**Версия решения:**
1. Логарифмирование таргета (`np.log1p`) для сглаживания выбросов.
2. Очистка весов от отрицательных значений (Fix CatBoostError).
3. Native NaN handling (оставляем пропуски для CatBoost, где это логично).
4. Регуляризация модели для предотвращения переобучения.

In [1]:
# 1. Установка библиотек
!pip install catboost shap pandas scikit-learn numpy

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import json
import os

# Метрика WMAE
def weighted_mean_absolute_error(y_true, y_pred, weights):
    return (weights * np.abs(y_true - y_pred)).mean()

In [3]:
# 2. Загрузка данных
print("Загрузка данных...")
try:
    train_df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';', low_memory=False)
    test_df = pd.read_csv('hackathon_income_test.csv', decimal=',', sep=';', low_memory=False)
    print(f"Train shape: {train_df.shape}")
    print(f"Test shape: {test_df.shape}")
except FileNotFoundError:
    print("ОШИБКА: Не найдены файлы csv. Загрузите их в сессию Colab!")

Загрузка данных...
Train shape: (76786, 224)
Test shape: (73214, 222)


In [4]:
# 3. Список признаков для удаления (Noise Features)
USELESS_FEATURES = [
    'addrref', 'city_smart_name', 'dp_ewb_last_employment_position',
    'client_active_flag', 'vert_has_app_ru_tinkoff_investing',
    'dp_ewb_dismissal_due_contract_violation_by_lb_cnt', 'period_last_act_ad',
    'ovrd_sum', 'businessTelSubs', 'dp_ils_days_ip_share_5y',
    'nonresident_flag', 'vert_has_app_ru_vtb_invest',
    'hdb_bki_total_pil_cnt', 'accountsalary_out_flag',
    'id', 'dt'
]

In [5]:
# 4. Функция предобработки (FINAL STABLE)
def preprocess_data(df, is_train=True):
    df_proc = df.copy()

    # 1. Удаляем явный мусор
    cols_to_drop = [c for c in USELESS_FEATURES if c in df_proc.columns]
    df_proc = df_proc.drop(columns=cols_to_drop, errors='ignore')

    # 2. Обработка текстовых чисел (очистка от пробелов)
    object_cols = df_proc.select_dtypes(include='object').columns
    for col in object_cols:
        if df_proc[col].nunique() > 50:
            try:
                temp_col = df_proc[col].astype(str).str.replace(' ', '').str.replace(',', '.')
                df_proc[col] = pd.to_numeric(temp_col, errors='coerce')
            except:
                pass

    # 3. Smart Features (Флаги пропусков)
    important_nans = ['salary_6to12m_avg', 'first_salary_income']
    for col in important_nans:
        if col in df_proc.columns:
            df_proc[f'{col}_is_missing'] = df_proc[col].isna().astype(int)

    # --- КРИТИЧЕСКОЕ ИСПРАВЛЕНИЕ: Очистка Таргета и Весов ---
    if is_train:
        # Принудительно в числа
        df_proc['target'] = pd.to_numeric(df_proc['target'], errors='coerce')
        df_proc['w'] = pd.to_numeric(df_proc['w'], errors='coerce')

        # 1. Удаляем строки, где target = NaN (на них нельзя учиться)
        initial_len = len(df_proc)
        df_proc = df_proc.dropna(subset=['target'])
        if len(df_proc) < initial_len:
            print(f"Удалено {initial_len - len(df_proc)} строк с пустым таргетом.")

        # 2. Убираем отрицательные значения (доход не может быть < 0)
        # Это спасет логарифм от NaN
        df_proc['target'] = df_proc['target'].clip(lower=0)

        # 3. Чистим веса (NaN -> 0, <0 -> 0)
        df_proc['w'] = df_proc['w'].fillna(0).clip(lower=0)

        # Выделяем y и w
        y = df_proc['target']
        w = df_proc['w']

        # Удаляем их из X
        df_proc = df_proc.drop(columns=['target', 'w'], errors='ignore')

    # 5. Заполнение пропусков в признаках
    # Группа А: Явные нули -> 0
    zero_fill_keywords = ['sum', 'count', 'cnt', 'amount', 'turn', 'limit', 'outstanding', 'balance']
    cols_to_zero = [c for c in df_proc.columns if any(k in c.lower() for k in zero_fill_keywords) and df_proc[c].dtype != 'object']
    df_proc[cols_to_zero] = df_proc[cols_to_zero].fillna(0)

    # Группа Б: Остальные -> Оставляем NaN (CatBoost справится)

    # Группа В: Категории -> 'MISSING'
    cat_cols = df_proc.select_dtypes(include=['object']).columns
    df_proc[cat_cols] = df_proc[cat_cols].fillna("MISSING")

    if is_train:
        return df_proc, y, w
    else:
        # На тесте просто удаляем колонки, если они есть
        df_proc = df_proc.drop(columns=['target', 'w'], errors='ignore')
        return df_proc

print("Препроцессинг данных (Target fixed)...")
X, y, w = preprocess_data(train_df, is_train=True)
X_submit = preprocess_data(test_df, is_train=False)

# Списки признаков для конфига
feature_names = list(X.columns)
cat_features = list(X.select_dtypes(include=['object']).columns)

print(f"Итоговое кол-во признаков: {len(feature_names)}")

Препроцессинг данных (Target fixed)...
Итоговое кол-во признаков: 208


In [6]:
# 5. Подготовка к обучению

# Split
X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(
    X, y, w, test_size=0.2, random_state=42
)

# Log-Target Transformation
# Учимся предсказывать логарифм дохода
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# Pools
train_pool = Pool(X_train, y_train_log, cat_features=cat_features, weight=w_train)
val_pool = Pool(X_val, y_val_log, cat_features=cat_features, weight=w_val)

In [7]:
# 6. Обучение модели
# Используем параметры для предотвращения переобучения

model = CatBoostRegressor(
    iterations=3000,
    learning_rate=0.03,
    depth=6,                  # Глубина 6 (оптимально для баланса bias/variance)
    l2_leaf_reg=10,           # Сильная регуляризация (было 3-5, стало 10)
    loss_function='RMSE',     # RMSE лучше для логарифмированного таргета
    eval_metric='MAE',
    random_seed=42,
    verbose=250,
    early_stopping_rounds=300,
    allow_writing_files=False,
    task_type="CPU"
)

print("Запуск обучения...")
model.fit(train_pool, eval_set=val_pool)

Запуск обучения...
0:	learn: 0.9454253	test: 0.9561922	best: 0.9561922 (0)	total: 288ms	remaining: 14m 25s
250:	learn: 0.4737170	test: 0.4885453	best: 0.4885453 (250)	total: 52.5s	remaining: 9m 34s
500:	learn: 0.4416560	test: 0.4622920	best: 0.4622920 (500)	total: 1m 44s	remaining: 8m 42s
750:	learn: 0.4223898	test: 0.4508268	best: 0.4508166 (748)	total: 2m 35s	remaining: 7m 45s
1000:	learn: 0.4108188	test: 0.4457175	best: 0.4457175 (1000)	total: 3m 24s	remaining: 6m 48s
1250:	learn: 0.4005178	test: 0.4419491	best: 0.4419491 (1250)	total: 4m 11s	remaining: 5m 51s
1500:	learn: 0.3912948	test: 0.4390655	best: 0.4390655 (1500)	total: 5m 3s	remaining: 5m 3s
1750:	learn: 0.3832259	test: 0.4371170	best: 0.4371132 (1749)	total: 5m 57s	remaining: 4m 15s
2000:	learn: 0.3756993	test: 0.4355206	best: 0.4355206 (2000)	total: 6m 47s	remaining: 3m 23s
2250:	learn: 0.3688377	test: 0.4344040	best: 0.4343872 (2241)	total: 7m 37s	remaining: 2m 32s
2500:	learn: 0.3621758	test: 0.4331515	best: 0.4331461 (

In [8]:
# 7. Оценка качества

log_preds = model.predict(X_val)
# Обратное преобразование: exp(x) - 1
real_preds = np.expm1(log_preds)
real_preds = np.maximum(real_preds, 0)

wmae_score = weighted_mean_absolute_error(y_val, real_preds, w_val)
print(f"==========================================")
print(f"FINAL Validation WMAE: {wmae_score:.2f}")
print(f"==========================================")

FINAL Validation WMAE: 38969.97


In [9]:
# 8. Важность признаков (для отчета)
feature_importances = model.get_feature_importance()
df_imp = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
df_imp = df_imp.sort_values(by='importance', ascending=False)
print(df_imp.head(10))

                        feature  importance
0        turn_cur_cr_avg_act_v2    6.100993
2       hdb_bki_total_max_limit    5.810847
6                        gender    4.137397
5                   incomeValue    3.839340
17       turn_cur_db_avg_act_v2    3.800453
1             salary_6to12m_avg    3.334818
11  hdb_bki_total_pil_max_limit    3.207794
4    hdb_bki_total_cc_max_limit    2.928706
12                          age    2.825265
44    per_capita_income_rur_amt    2.769479


In [10]:
# 9. Сохранение результатов

# A. Предсказание на тесте для сабмита
log_submit_preds = model.predict(X_submit)
submit_preds = np.expm1(log_submit_preds)
submit_preds = np.maximum(submit_preds, 0)

submission = test_df[['id']].copy()
submission['target'] = submit_preds
submission.to_csv('submission_final.csv', index=False)
print("1. Файл submission_final.csv готов.")

# B. Сохранение модели
model.save_model("model.cbm")
print("2. Файл model.cbm сохранен.")

# C. Сохранение конфига признаков
metadata = {
    "feature_names": feature_names,
    "cat_features": cat_features
}
with open("features.json", "w") as f:
    json.dump(metadata, f)
print("3. Файл features.json сохранен.")

1. Файл submission_final.csv готов.
2. Файл model.cbm сохранен.
3. Файл features.json сохранен.
